# Getting The Data

In [1]:
import pandas as pd
ghi_df = pd.read_csv('/content/drive/MyDrive/predicted_ghi.csv')
print(ghi_df)

    hour  predicted_ghi        date
0      0       0.000000  2025-10-09
1      1       0.000000  2025-10-09
2      2       0.000000  2025-10-09
3      3       0.000000  2025-10-09
4      4       0.000000  2025-10-09
5      5       0.000000  2025-10-09
6      6       0.000000  2025-10-09
7      7      65.597340  2025-10-09
8      8     174.463470  2025-10-09
9      9     298.220580  2025-10-09
10    10     443.440770  2025-10-09
11    11     560.139600  2025-10-09
12    12     639.329400  2025-10-09
13    13     648.836400  2025-10-09
14    14     606.947630  2025-10-09
15    15     491.622770  2025-10-09
16    16     326.187070  2025-10-09
17    17     157.321370  2025-10-09
18    18      38.067554  2025-10-09
19    19       0.000000  2025-10-09
20    20       0.000000  2025-10-09
21    21       0.000000  2025-10-09
22    22       0.000000  2025-10-09
23    23       0.000000  2025-10-09


In [2]:
import pandas as pd
load_df = pd.read_csv('/content/drive/MyDrive/predicted_load.csv')
print(load_df)

    hour  predicted_load        date
0      0       7685.0930  2025-10-09
1      1       7156.7070  2025-10-09
2      2       6732.7060  2025-10-09
3      3       6442.0140  2025-10-09
4      4       6328.8784  2025-10-09
5      5       6341.0220  2025-10-09
6      6       6510.8657  2025-10-09
7      7       6838.9023  2025-10-09
8      8       7046.9490  2025-10-09
9      9       7328.2065  2025-10-09
10    10       7572.7905  2025-10-09
11    11       7843.7420  2025-10-09
12    12       8167.2134  2025-10-09
13    13       8484.8960  2025-10-09
14    14       8830.7280  2025-10-09
15    15       9254.6880  2025-10-09
16    16       9659.0670  2025-10-09
17    17      10019.4470  2025-10-09
18    18      10153.8430  2025-10-09
19    19       9991.4900  2025-10-09
20    20       9671.2020  2025-10-09
21    21       9373.9290  2025-10-09
22    22       8919.2650  2025-10-09
23    23       8334.5970  2025-10-09


# To Find PV and Battery Sizes

In [91]:
battery_capacity_list = []
final_PV_plant_m2_list = []

def simulate_day(target_date, seasonal_load_df, seasonal_ghi_df):

  load_day = seasonal_load_df[seasonal_load_df['date'] == target_date].reset_index(drop=True)
  ghi_day = seasonal_ghi_df[seasonal_ghi_df['date'] == target_date].reset_index(drop=True)

  '''Finding the Final Solar Photovoltaic Plant in m²'''
  PV_with_bonus = 0.20 * 1.31
  PR = 0.786
  daily_load_Wh = sum(load_day.iloc[:,1]) * 1_000_000
  daily_GHI_Wh_m2 = sum(ghi_day.iloc[:,1])

  final_PV_plant_m2 = daily_load_Wh / (daily_GHI_Wh_m2 * PV_with_bonus * PR)
  final_PV_plant_m2_list.append(final_PV_plant_m2)


  print(f'Solar PV Plant Size {final_PV_plant_m2:} m²\n')


  '''Hourly Solar Photovoltaic Output & Surplus/Shortage Calculations'''

  total_days_shortage = 0
  total_days_surplus = 0

  battery_lowest = 0
  lowest_values = []

  for hour in range(24):
    load_for_the_hour_Wh = (load_day.iloc[hour,1]) * 1000000
    hour_GHI_Wh_m2 = (ghi_day.iloc[hour,1])

    if hour_GHI_Wh_m2 > 0:
      hour_PV_output_MWh = (hour_GHI_Wh_m2 * PV_with_bonus * PR * final_PV_plant_m2) / 1000000
      diff = hour_PV_output_MWh - (load_for_the_hour_Wh / 1000000)

      if diff >= 0:
        total_days_surplus += diff
      else:
        total_days_shortage += abs(diff)

      print(
          f"{hour}: Solar PV output: {hour_PV_output_MWh} MWh & "
          f"Actual load: {load_for_the_hour_Wh / 1000000} MWh & "
          f"{'Surplus of ' if diff >= 0 else 'Shortage of -'}{abs(diff)} MWh"
      )

      battery_lowest += diff
      lowest_values.append(battery_lowest)

    else:
      print(f'{hour}: Shortage of -{load_for_the_hour_Wh / 1000000} MWh') # shortage
      total_days_shortage += load_for_the_hour_Wh / 1000000
      battery_lowest +=  -load_for_the_hour_Wh / 1000000
      lowest_values.append(battery_lowest)

  print(f'\nSurplus: {total_days_surplus} MWh in Solar PV going to Battery Storage')
  print(f'Shortage: {total_days_shortage} MWh either accessed from Battery Storage or NGCC')
  print(lowest_values)

  '''Capacity of Battery'''
  battery_capacity = abs(min(lowest_values))
  battery_capacity_list.append(battery_capacity)

  print(battery_capacity)

In [92]:
''' Load Data for Representative Dates'''

import pandas as pd
import os

# Path to your seasonal file
save_directory = '/content/drive/MyDrive'
seasonal_file = os.path.join(save_directory, 'seasonal_predicted_loads.csv')

# Check if the file exists and is non-empty
if os.path.exists(seasonal_file) and os.path.getsize(seasonal_file) > 0:
    try:
        seasonal_load_df = pd.read_csv(seasonal_file)
        print("✅ Seasonal data loaded successfully!")
        print(seasonal_load_df)
    except pd.errors.EmptyDataError:
        print("⚠️ Seasonal file exists but is empty.")
        seasonal_load_df = pd.DataFrame()  # create empty DataFrame
else:
    print("⚠️ Seasonal file does not exist or is empty.")
    seasonal_load_df = pd.DataFrame()  # create empty DataFrame


✅ Seasonal data loaded successfully!
     hour  predicted_load        date
0       0     6730.208000  2025-03-21
1       1     6317.309600  2025-03-21
2       2     5992.536000  2025-03-21
3       3     5779.637700  2025-03-21
4       4     5609.925300  2025-03-21
..    ...             ...         ...
283    19     8018.749023  2025-02-21
284    20     8250.614258  2025-02-21
285    21     8259.625977  2025-02-21
286    22     8042.062500  2025-02-21
287    23     7776.392090  2025-02-21

[288 rows x 3 columns]


In [93]:
''' Solar GHI Data for Representative Dates'''

import pandas as pd
import os

# Path to your seasonal file
save_directory = '/content/drive/MyDrive'
GHI_file = os.path.join(save_directory, 'seasonal_predicted_ghi.csv')

# Check if the file exists and is non-empty
if os.path.exists(GHI_file) and os.path.getsize(GHI_file) > 0:
    try:
        seasonal_ghi_df = pd.read_csv(GHI_file)
        print("✅ GHI data loaded successfully!")
        print(seasonal_ghi_df)
    except pd.errors.EmptyDataError:
        print("⚠️ GHI file exists but is empty.")
        seasonal_ghi_df = pd.DataFrame()  # create empty DataFrame
else:
    print("⚠️ GHI file does not exist or is empty.")
    seasonal_ghi_df = pd.DataFrame()  # create empty DataFrame


✅ GHI data loaded successfully!
     hour  predicted_ghi        date
0       0            0.0  2025-03-21
1       1            0.0  2025-03-21
2       2            0.0  2025-03-21
3       3            0.0  2025-03-21
4       4            0.0  2025-03-21
..    ...            ...         ...
283    19            0.0  2025-02-21
284    20            0.0  2025-02-21
285    21            0.0  2025-02-21
286    22            0.0  2025-02-21
287    23            0.0  2025-02-21

[288 rows x 3 columns]


In [94]:
simulate_day("2025-03-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-04-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-05-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 186478332.00835058 m²

0: Shortage of -6730.208 MWh
1: Shortage of -6317.3096 MWh
2: Shortage of -5992.536 MWh
3: Shortage of -5779.6377 MWh
4: Shortage of -5609.9253 MWh
5: Shortage of -5574.014 MWh
6: Shortage of -5655.4165 MWh
7: Solar PV output: 622.8279024734177 MWh & Actual load: 5845.335 MWh & Shortage of -5222.507097526583 MWh
8: Solar PV output: 4844.0818564556075 MWh & Actual load: 6086.4697 MWh & Shortage of -1242.3878435443921 MWh
9: Solar PV output: 8154.8230087677675 MWh & Actual load: 6405.8887 MWh & Surplus of 1748.934308767767 MWh
10: Solar PV output: 11872.922617242399 MWh & Actual load: 6707.161 MWh & Surplus of 5165.761617242399 MWh
11: Solar PV output: 14882.075073421678 MWh & Actual load: 6735.9727 MWh & Surplus of 8146.102373421678 MWh
12: Solar PV output: 19930.25146123223 MWh & Actual load: 6783.007 MWh & Surplus of 13147.24446123223 MWh
13: Solar PV output: 21620.40657976773 MWh & Actual load: 6813.663 MWh & Surplus of 14806.743579767728 MW

In [95]:
simulate_day("2025-06-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-07-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-08-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 158689228.93585175 m²

0: Shortage of -10040.7861328125 MWh
1: Shortage of -9209.10546875 MWh
2: Shortage of -8607.013671875 MWh
3: Shortage of -8225.138671875 MWh
4: Shortage of -7970.302734375 MWh
5: Shortage of -7622.4267578125 MWh
6: Shortage of -7478.98486328125 MWh
7: Solar PV output: 2812.4276330440525 MWh & Actual load: 7438.9345703125 MWh & Shortage of -4626.5069372684475 MWh
8: Solar PV output: 7735.0435706168 MWh & Actual load: 7602.40087890625 MWh & Surplus of 132.64269171055003 MWh
9: Solar PV output: 13090.134953137262 MWh & Actual load: 8006.681640625 MWh & Surplus of 5083.453312512262 MWh
10: Solar PV output: 20157.509496206778 MWh & Actual load: 8403.2412109375 MWh & Surplus of 11754.268285269278 MWh
11: Solar PV output: 24960.215553582453 MWh & Actual load: 9075.12109375 MWh & Surplus of 15885.094459832453 MWh
12: Solar PV output: 28078.277836986876 MWh & Actual load: 9823.2373046875 MWh & Surplus of 18255.040532299376 MWh
13: Solar PV output: 2889

In [96]:
simulate_day("2025-09-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-10-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-11-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 189546653.38424224 m²

0: Shortage of -8827.7314453125 MWh
1: Shortage of -8146.84375 MWh
2: Shortage of -7746.16064453125 MWh
3: Shortage of -7347.4169921875 MWh
4: Shortage of -7165.83984375 MWh
5: Shortage of -7210.74951171875 MWh
6: Shortage of -7434.9755859375 MWh
7: Solar PV output: 878.3162826137292 MWh & Actual load: 7739.96875 MWh & Shortage of -6861.6524673862705 MWh
8: Solar PV output: 5741.92379419588 MWh & Actual load: 7886.7626953125 MWh & Shortage of -2144.83890111662 MWh
9: Solar PV output: 12562.550817225903 MWh & Actual load: 8241.267578125 MWh & Surplus of 4321.283239100903 MWh
10: Solar PV output: 18340.78998103098 MWh & Actual load: 8642.4580078125 MWh & Surplus of 9698.33197321848 MWh
11: Solar PV output: 27501.97747796271 MWh & Actual load: 9059.1083984375 MWh & Surplus of 18442.86907952521 MWh
12: Solar PV output: 31626.29373000586 MWh & Actual load: 9587.666015625 MWh & Surplus of 22038.62771438086 MWh
13: Solar PV output: 31383.000051301467

In [97]:
simulate_day("2025-12-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-01-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-02-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 318363810.0194176 m²

0: Shortage of -6590.6796875 MWh
1: Shortage of -6419.68359375 MWh
2: Shortage of -6270.0166015625 MWh
3: Shortage of -6161.58837890625 MWh
4: Shortage of -6253.07177734375 MWh
5: Shortage of -6415.90869140625 MWh
6: Shortage of -6880.91845703125 MWh
7: Solar PV output: 792.7580021964432 MWh & Actual load: 7534.10791015625 MWh & Shortage of -6741.349907959807 MWh
8: Solar PV output: 7143.825368613846 MWh & Actual load: 7992.43896484375 MWh & Shortage of -848.613596229904 MWh
9: Solar PV output: 13524.272195781961 MWh & Actual load: 7997.8076171875 MWh & Surplus of 5526.464578594461 MWh
10: Solar PV output: 19118.564693710217 MWh & Actual load: 7715.7314453125 MWh & Surplus of 11402.833248397717 MWh
11: Solar PV output: 24459.511440890128 MWh & Actual load: 7430.4833984375 MWh & Surplus of 17029.028042452628 MWh
12: Solar PV output: 31635.02203490932 MWh & Actual load: 7292.4482421875 MWh & Surplus of 24342.57379272182 MWh
13: Solar PV output: 3

In [98]:
final_battery_size = sum(battery_capacity_list) / len(battery_capacity_list)
final_PV_plant_size = sum(final_PV_plant_m2_list) / len(final_PV_plant_m2_list)

print(f"Final Averaged Battery Capacity: {final_battery_size}")
print(f"Final Averaged PV Plant Size: {final_PV_plant_size}")

Final Averaged Battery Capacity: 58220.14704667093
Final Averaged PV Plant Size: 214271139.90200898


Final Averaged Battery Capacity: 57706.92400050562
Final Averaged PV Plant Size: 1304564487.4166667


# Calculating TODAY'S Information

In [99]:
'''Today's Hourly Solar PV Output & Surplus/Shortage Calculations'''

amt_in_battery = 0

for hour in range(24):
  load_for_the_hour_Wh = (load_df.iloc[hour,1]) * 1000000
  hour_GHI_Wh_m2 = (ghi_df.iloc[hour,1])

  PV_with_bonus = 0.20 * 1.31
  PR = 0.786

  if hour_GHI_Wh_m2 > 0:
    hour_PV_output_MWh = (hour_GHI_Wh_m2 * PV_with_bonus * PR * final_PV_plant_size) / 1000000
    diff = hour_PV_output_MWh - (load_for_the_hour_Wh / 1000000)
    print(
        f"{hour}: The Solar PV output: {hour_PV_output_MWh} MWh, "
        f"Actual load: {load_for_the_hour_Wh / 1000000} MWh, "
        f"{'Surplus of ' if diff >= 0 else 'Shortage of -'}{abs(diff)} MWh"
    )
    if diff >= 0:
      if amt_in_battery < final_battery_size:
        amt_in_battery += (diff)

  else:
    print(f'{hour}: Shortage of -{load_for_the_hour_Wh / 1000000} MWh')
    amt_in_battery -= (load_for_the_hour_Wh / 1000000)
    # RUN ABOVEEEE

print()
print(f"{amt_in_battery} MWh in the Battery Storage")

0: Shortage of -7685.093 MWh
1: Shortage of -7156.707 MWh
2: Shortage of -6732.706 MWh
3: Shortage of -6442.014 MWh
4: Shortage of -6328.8784 MWh
5: Shortage of -6341.022 MWh
6: Shortage of -6510.8657 MWh
7: The Solar PV output: 2894.5012822224576 MWh, Actual load: 6838.9023 MWh, Shortage of -3944.401017777542 MWh
8: The Solar PV output: 7698.250228072955 MWh, Actual load: 7046.949 MWh, Surplus of 651.3012280729554 MWh
9: The Solar PV output: 13159.067901154602 MWh, Actual load: 7328.2065 MWh, Surplus of 5830.8614011546015 MWh
10: The Solar PV output: 19566.950082956315 MWh, Actual load: 7572.7905 MWh, Surplus of 11994.159582956316 MWh
11: The Solar PV output: 24716.319143788056 MWh, Actual load: 7843.742 MWh, Surplus of 16872.577143788054 MWh
12: The Solar PV output: 28210.59158896556 MWh, Actual load: 8167.2134 MWh, Surplus of 20043.37818896556 MWh
13: The Solar PV output: 28630.09066758809 MWh, Actual load: 8484.896 MWh, Surplus of 20145.19466758809 MWh
14: The Solar PV output: 2678

In [100]:
'''Today's Hourly Solar PV Output & Surplus/Shortage Calculations'''

amt_in_battery = 0
fossil_used = 0
solar_wasted = 0

for hour in range(24):
    load_for_the_hour_Wh = (load_df.iloc[hour, 1]) * 1000000
    hour_GHI_Wh_m2 = (ghi_df.iloc[hour, 1])

    PV_with_bonus = 0.20 * 1.31
    PR = 0.786

    if hour_GHI_Wh_m2 > 0:
        hour_PV_output_MWh = (hour_GHI_Wh_m2 * PV_with_bonus * PR * final_PV_plant_size) / 1000000
        diff = hour_PV_output_MWh - (load_for_the_hour_Wh / 1000000)

        print(
            f"{hour}: The Solar PV output: {hour_PV_output_MWh} MWh, "
            f"Actual load: {load_for_the_hour_Wh / 1000000} MWh, "
            f"{'Surplus of ' if diff >= 0 else 'Shortage of -'}{abs(diff)} MWh"
        )

        if diff >= 0:  # surplus → store in battery
            available_space = final_battery_size - amt_in_battery
            charge = min(diff, available_space)
            amt_in_battery += charge
            solar_wasted += max(0, diff - charge)  # if battery full, waste rest

        else:  # shortage → draw from battery
            needed = abs(diff)
            discharge = min(needed, amt_in_battery)
            amt_in_battery -= discharge
            fossil_used += max(0, needed - discharge)  # if battery empty, use fossil
    else:
        shortage = load_for_the_hour_Wh / 1000000
        print(f"{hour}: Shortage of -{shortage} MWh")

        discharge = min(shortage, amt_in_battery)
        amt_in_battery -= discharge
        fossil_used += max(0, shortage - discharge)

print()
print(f"{amt_in_battery} MWh remaining in the Battery Storage")
print(f"{solar_wasted} MWh of solar energy wasted (battery full)")
print(f"{fossil_used} MWh supplied by fossil fuels (battery empty)")


0: Shortage of -7685.093 MWh
1: Shortage of -7156.707 MWh
2: Shortage of -6732.706 MWh
3: Shortage of -6442.014 MWh
4: Shortage of -6328.8784 MWh
5: Shortage of -6341.022 MWh
6: Shortage of -6510.8657 MWh
7: The Solar PV output: 2894.5012822224576 MWh, Actual load: 6838.9023 MWh, Shortage of -3944.401017777542 MWh
8: The Solar PV output: 7698.250228072955 MWh, Actual load: 7046.949 MWh, Surplus of 651.3012280729554 MWh
9: The Solar PV output: 13159.067901154602 MWh, Actual load: 7328.2065 MWh, Surplus of 5830.8614011546015 MWh
10: The Solar PV output: 19566.950082956315 MWh, Actual load: 7572.7905 MWh, Surplus of 11994.159582956316 MWh
11: The Solar PV output: 24716.319143788056 MWh, Actual load: 7843.742 MWh, Surplus of 16872.577143788054 MWh
12: The Solar PV output: 28210.59158896556 MWh, Actual load: 8167.2134 MWh, Surplus of 20043.37818896556 MWh
13: The Solar PV output: 28630.09066758809 MWh, Actual load: 8484.896 MWh, Surplus of 20145.19466758809 MWh
14: The Solar PV output: 2678

# STEPS...

1) *Pick representative dates for each season (or better: pick a small set of 3–5 representative days per season).*
2) *For each chosen date (or each day in a small seasonal sample) compute the hourly diff = PV − load, the cumulative running tally, and the battery capacity = abs(min(running_tally)).*
3) Apply the 0.0001 convergence idea: instead of adding one hour at a time (your professor’s suggestion), it’s more meaningful with seasonal sampling to add days (or if you prefer, add hours across many days). For each season:

* Start with 1 day, compute the average of the battery requirements (or average
of absolute diffs), then add one more day from the same season, recompute the average, and stop when relative change < 0.0001.
* This tells you the number of days needed from that season to reach a stable average.

^^By Friday

---

Combine seasons into a final design. Options:

* Conservative: choose the maximum battery size across seasons (safe but costly).

* Weighted: weight each season by the number of days it represents (e.g., 90 days per season) and compute a weighted average battery requirement.

* Risk-based: pick the 95th percentile battery size across all sampled days (trades cost vs. reliability).

^^By Sunday


**Representative Dates**

Spring: March 21, **April 21**, May 21

Summer: June 21, **July 21**, August 21

Fall: September 21, **October 21**, November 21

Winter: December 21, **January 21**, February 21

In [ ]:
battery_capacity_list = []
final_PV_plant_m2_list = []

def simulate_day(target_date, seasonal_load_df, seasonal_ghi_df):

  load_day = seasonal_load_df[seasonal_load_df['date'] == target_date].reset_index(drop=True)
  ghi_day = seasonal_ghi_df[seasonal_ghi_df['date'] == target_date].reset_index(drop=True)

  '''Finding the Final Solar Photovoltaic Plant in m²'''
  PV_plant_sizes_m2 = []
  for hour in range(24):

    load_for_the_hour_Wh = (load_day.iloc[hour,1]) * 1000000
    hour_GHI_Wh_m2 = (ghi_day.iloc[hour,1])
    PV_with_bonus = 0.20 * 1.31
    PR = 0.786

    if hour_GHI_Wh_m2 > 0:
      plant_size_m2 = (load_for_the_hour_Wh) / (hour_GHI_Wh_m2 * PV_with_bonus * PR)
      PV_plant_sizes_m2.append(plant_size_m2)

  final_PV_plant_m2 = 0
  for x in PV_plant_sizes_m2:
    final_PV_plant_m2 += x
  final_PV_plant_m2 = final_PV_plant_m2 // (len(PV_plant_sizes_m2))
  final_PV_plant_m2_list.append(final_PV_plant_m2)

  print(f'Solar PV Plant Size {final_PV_plant_m2:} m²\n')


  '''Hourly Solar Photovoltaic Output & Surplus/Shortage Calculations'''

  total_days_shortage = 0
  total_days_surplus = 0

  battery_lowest = 0
  lowest_values = []

  for hour in range(24):
    load_for_the_hour_Wh = (load_day.iloc[hour,1]) * 1000000
    hour_GHI_Wh_m2 = (ghi_day.iloc[hour,1])

    if hour_GHI_Wh_m2 > 0:
      hour_PV_output_MWh = (hour_GHI_Wh_m2 * PV_with_bonus * PR * final_PV_plant_m2) / 1000000
      diff = hour_PV_output_MWh - (load_for_the_hour_Wh / 1000000)

      if diff >= 0:
        total_days_surplus += diff
      else:
        total_days_shortage += abs(diff)

      print(
          f"{hour}: Solar PV output: {hour_PV_output_MWh} MWh & "
          f"Actual load: {load_for_the_hour_Wh / 1000000} MWh & "
          f"{'Surplus of ' if diff >= 0 else 'Shortage of -'}{abs(diff)} MWh"
      )

      battery_lowest += diff
      lowest_values.append(battery_lowest)

    else:
      print(f'{hour}: Shortage of -{load_for_the_hour_Wh / 1000000} MWh') # shortage
      total_days_shortage += load_for_the_hour_Wh / 1000000
      battery_lowest +=  -load_for_the_hour_Wh / 1000000
      lowest_values.append(battery_lowest)

  print(f'\nSurplus: {total_days_surplus} MWh in Solar PV going to Battery Storage')
  print(f'Shortage: {total_days_shortage} MWh either accessed from Battery Storage or NGCC')
  print(lowest_values)

  '''Capacity of Battery'''
  battery_capacity = abs(sum(lowest_values) / len(lowest_values))
  battery_capacity_list.append(battery_capacity)

  print(battery_capacity)